In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline

In [13]:
# car_id: [1, ..., 142, ..., 272]
df = pd.read_csv('2020_02_25.csv', 
    dtype={'total_cars':int, 'timestamp':object, 'cars_list':str, 'latitude':float, 'longitude':float}, 
    parse_dates=['timestamp'], 
    infer_datetime_format=True)
df.head(5)

,latitude,longitude,total_cars,cars_list,timestamp
0,32.09004,34.79679,1,[138],2020-02-25 10:48:02+00:00
1,32.11614,34.79464,1,[64],2020-02-25 10:48:02+00:00
2,32.11876,34.82229,1,[120],2020-02-25 10:48:02+00:00
3,32.11226,34.83799,1,[110],2020-02-25 10:48:02+00:00
4,32.14279,34.79343,1,[190],2020-02-25 10:48:02+00:00


In [3]:
df = df.loc[df.total_cars > 0]
# df.timestamp = df.timestamp.apply(pd.Timestamp)
# df.head(5)

In [4]:
import json

car_list = df['cars_list'].apply(json.loads)

In [6]:
total_cars_list = []
for cars in car_list:
    total_cars_list += cars

In [7]:
total_cars_list = np.array(total_cars_list)

In [9]:
total_cars_list.shape

(12837542,)

In [10]:
import folium
from folium.plugins import HeatMap

In [11]:
start = df.timestamp.min() + pd.Timedelta("10min")
df_locations = df.loc[(start < df.timestamp) & (df.timestamp < start + pd.Timedelta("20min"))].groupby(['latitude', 'longitude', 'timestamp']).sum().reset_index()
len(df_locations) 

TypeError: can only concatenate str (not "Timedelta") to str

In [ ]:
import folium
from folium.plugins import HeatMap

center = [32.085, 34.800]
m = folium.Map(center, zoom_start=12)

# plot grid
# max
# latitude_min = 32.030
# latitude_max = 32.145 # 0.115
# longitude_min = 34.745
# longitude_max = 34.845 # 0.100

# sample
latitude_min = 32.045
longitude_min = 34.76

latitude_step = 0.0027 # 300m
longitude_step = 0.00318 # 300m

latitude_max = latitude_min + 15*latitude_step
longitude_max = longitude_min + 15*longitude_step

locations = [[latitude_min, longitude_min], [latitude_min, longitude_max],
            [latitude_max, longitude_max], [latitude_max, longitude_min],
            [latitude_min, longitude_min]]

folium.PolyLine(
    locations, # 坐标点列表
    weight=2, # 线宽
    color='black', # 线条颜色
    opacity=0.8 # 透明度
).add_to(m)

latitude_num = int((latitude_max - latitude_min)/latitude_step)
longitude_num = int((longitude_max - longitude_min)/longitude_step)
polygons = []
for i in range(latitude_num):
    latitude_right = latitude_min + latitude_step * (i+1)
    polygons.append([[latitude_right, longitude_min], [latitude_right, longitude_max]])

for i in range(longitude_num):
    longitude_down  = longitude_min + longitude_step * (i+1)
    polygons.append([[latitude_min, longitude_down], [latitude_max, longitude_down]])

folium.PolyLine(
    polygons, # 坐标点列表
    weight=1, # 线宽
    color='black', # 线条颜色
    opacity=0.8 # 透明度
).add_to(m)

df_locations = df.loc[df.timestamp == time[220]].groupby(['latitude', 'longitude', 'timestamp']).sum().reset_index()
for index, row in df_locations.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=row['total_cars'] * 6,
                        fill_color="#3db7e4", 
                    ).add_to(m)
    
points = df_locations[['latitude', 'longitude']].values
m.add_children(HeatMap(points, radius=15)) # plot heatmap
m

In [ ]:
df.loc[df.timestamp == time[2]]

,latitude,longitude,total_cars,cars_list,timestamp
13510251,32.033860,34.758440,1,[59],2018-07-04 00:14:28 UTC
13510252,32.048050,34.782830,1,[81],2018-07-04 00:14:28 UTC
13510253,32.098600,34.788400,1,[90],2018-07-04 00:14:28 UTC
13510254,32.101180,34.783070,1,[97],2018-07-04 00:14:28 UTC
13510255,32.083266,34.791986,1,[108],2018-07-04 00:14:28 UTC
...,...,...,...,...,...
13510570,32.121290,34.799330,1,[271],2018-07-04 00:14:28 UTC
13510571,32.096600,34.773800,2,"[22, 267]",2018-07-04 00:14:28 UTC
13510572,32.084900,34.796600,2,"[169, 205]",2018-07-04 00:14:28 UTC
13510573,32.098032,34.798089,2,"[215, 250]",2018-07-04 00:14:28 UTC


In [ ]:
time = df['timestamp'].drop_duplicates().sort_values().reset_index(drop=True)
time.head(100)

0     2018-07-04 00:04:24 UTC
1     2018-07-04 00:09:26 UTC
2     2018-07-04 00:14:28 UTC
3     2018-07-04 00:19:30 UTC
4     2018-07-04 00:24:33 UTC
               ...           
95    2018-07-04 08:02:34 UTC
96    2018-07-04 08:07:35 UTC
97    2018-07-04 08:12:38 UTC
98    2018-07-04 08:17:39 UTC
99    2018-07-04 08:22:42 UTC
Name: timestamp, Length: 100, dtype: object

In [ ]:
df.loc[0]

latitude                     32.09004
longitude                    34.79679
total_cars                          1
cars_list                       [138]
timestamp     2020-02-25 10:48:02 UTC
Name: 0, dtype: object